In [ ]:
# hide
# skip
!git clone https://github.com/benihime91/gale # install gale on colab
!pip install -e "gale[dev]"

In [ ]:
# default_exp utils.logger

In [ ]:
# hide
%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
# export
# hide
"""
Modified from :
https://github.com/facebookresearch/detectron2/blob/9c7f8a142216ebc52d3617c11f8fafd75b74e637/detectron2/utils/logger.py#L119
"""

'\nModified from :\nhttps://github.com/facebookresearch/detectron2/blob/9c7f8a142216ebc52d3617c11f8fafd75b74e637/detectron2/utils/logger.py#L119\n'

In [ ]:
# hide
import warnings
from nbdev.export import *
from nbdev.showdoc import *

warnings.filterwarnings("ignore")

<IPython.core.display.Javascript object>

# Logger
> Setups `logger`: name, level, format etc.

In [ ]:
# export
import functools
import logging
import sys

from fastcore.all import ifnone
from pytorch_lightning.utilities import rank_zero_only
from termcolor import colored

<IPython.core.display.Javascript object>

In [ ]:
# export
class _ColorfulFormatter(logging.Formatter):
    def __init__(self, *args, **kwargs):
        self._root_name = kwargs.pop("root_name") + "."
        self._abbrev_name = kwargs.pop("abbrev_name", "")
        if len(self._abbrev_name):
            self._abbrev_name = self._abbrev_name + "."
        super(_ColorfulFormatter, self).__init__(*args, **kwargs)

    def formatMessage(self, record):
        record.name = record.name.replace(self._root_name, self._abbrev_name)
        log = super(_ColorfulFormatter, self).formatMessage(record)
        if record.levelno == logging.WARNING:
            prefix = colored("WARNING", "red", attrs=["blink"])
        elif record.levelno == logging.ERROR or record.levelno == logging.CRITICAL:
            prefix = colored("ERROR", "red", attrs=["blink", "underline"])
        else:
            return log
        return prefix + " " + log

<IPython.core.display.Javascript object>

In [ ]:
# export
@functools.lru_cache()  # so that calling setup_logger multiple times won't add many handlers
def setup_logger(distributed_rank=0, *, color=True, name="gale", level=logging.DEBUG):
    """
    Initialize the gale logger and set its verbosity level to `level`.
    """
    logger = logging.getLogger(name)
    logger.setLevel(level)
    logger.propagate = False

    abbrev_name = name

    plain_formatter = logging.Formatter(
        "[%(asctime)s] %(name)s %(levelname)s: %(message)s", datefmt="%m/%d %H:%M:%S"
    )

    # stdout logging: master only
    if distributed_rank == 0:
        ch = logging.StreamHandler(stream=sys.stdout)
        ch.setLevel(logging.DEBUG)
        if color:
            formatter = _ColorfulFormatter(
                colored("[%(asctime)s %(name)s]: ", "green") + "%(message)s",
                datefmt="%m/%d %H:%M:%S",
                root_name=name,
                abbrev_name=str(abbrev_name),
            )
        else:
            formatter = plain_formatter
        ch.setFormatter(formatter)
        logger.addHandler(ch)
    return logger

<IPython.core.display.Javascript object>

In [ ]:
setup_logger()

logger = logging.getLogger("gale.utils.logger")

logger.info("This is a INFO message")
logger.debug("This is a DEBUG message")
logger.warning("This is a WARNING message")
logger.error("This is a ERROR message")

[04/30 16:16:47 gale.utils.logger]: This is a INFO message
[04/30 16:16:47 gale.utils.logger]: This is a DEBUG message
WARNING [04/30 16:16:47 gale.utils.logger]: This is a WARNING message
ERROR [04/30 16:16:47 gale.utils.logger]: This is a ERROR message


<IPython.core.display.Javascript object>

In [ ]:
# export
@rank_zero_only
def log_main_process(logger, lvl, msg):
    """
    Logs `msg` using `logger` only on the main process
    """
    logger.log(lvl, msg)

<IPython.core.display.Javascript object>

In [ ]:
log_main_process(logger, logging.INFO, "This logs only on the main process")
log_main_process(logger, logging.ERROR, "This logs only on the main process")
log_main_process(logger, logging.WARNING, "This logs only on the main process")

[04/30 16:16:52 gale.utils.logger]: This logs only on the main process
ERROR [04/30 16:16:52 gale.utils.logger]: This logs only on the main process
WARNING [04/30 16:16:52 gale.utils.logger]: This logs only on the main process


<IPython.core.display.Javascript object>

## Export -

In [ ]:
# hide
notebook2script("00_utils.logger.ipynb")

Converted 00_utils.logger.ipynb.


<IPython.core.display.Javascript object>